In [56]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd 

sheet = pd.read_csv('training.csv',encoding = "ISO-8859-1" , names=['note','id','date','query','user','review'])

sheet.head()

,note,id,date,query,user,review
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [57]:
sheet = sheet.drop(columns=['id','date','query','user'])

In [58]:
sheet.shape

(1600000, 2)

In [59]:
sheet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
note      1600000 non-null int64
review    1600000 non-null object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [60]:
sheet.note.value_counts()

4    800000
0    800000
Name: note, dtype: int64

In [70]:
from sklearn.utils import shuffle

data = pd.DataFrame()



for k in range(2):
    note = k*4
    df = sheet[sheet.note == note]
    data = data.append( df.sample(n=30000) )
    
data.note = data.note.apply(lambda x: 1 if x==4 else 0)

data = shuffle(data)
data.head()

,note,review
1345269,1,Just got a ticket to go hear John Piper at @ad...
1539357,1,"Oh, look who is on lunch."
1434375,1,YAY!! this gig is almost over. only and hour a...
262889,0,@emmylucille Welcome! I'm kind of bad about tw...
1519936,1,@chrissie_ I MEANT U PWNED HIM!!!! hes just a ...


In [71]:
#rt
data.review = data.review.str.replace(r'(RT|rt)[ ]*@[ ]*[\S]+',r'')

#whitespaces
data.review = data.review.apply(lambda x: x.replace('\n', ' '))

#lowercase
data.review = data.review.apply(lambda x: x.lower())

#url
data.review = data.review.str.replace('http\S+|www.\S+', '', case=False)

# @ sign
data.review = data.review.str.replace(r'@[\S]+',r'')

In [72]:
import emoji

def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

data.review = data.review.apply(lambda x: deEmojify(x))

In [73]:
import string

punct =[]
punct += list(string.punctuation)
punct += '’'
punct.remove("\"")
punct.remove("!")
punct.remove("?")

# punct

In [74]:
def remove_punctuations(text):
    for punctuation in punct:
        text = text.replace(punctuation, ' ')
    return text

In [75]:
data.review = data.review.apply(lambda x: remove_punctuations(x))

In [76]:
# < and >
data.review = data.review.str.replace(r'&lt;',r'<')
data.review = data.review.str.replace(r'&gt;',r'>')

In [77]:
# insert space between ! or ? and text  
data.review = data.review.str.replace(r'([\w\d]+)([^\w\d ]+)', r'\1 \2')
data.review = data.review.str.replace(r'([^\w\d ]+)([\w\d]+)', r'\1 \2')

In [78]:
long = data.review.apply(lambda x : len(x.split(' ')))
max(long)

79

In [79]:
id =[]
for index, row in data.iterrows():
    if len(str(row.review).split(' ')) > 50:
        id.append(index)
        
data.drop(id, inplace = True) 
print(data.shape)

(59989, 2)


In [80]:
data.review

1345269    just got a ticket to go hear john piper at  to...
1539357                           oh  look who is on lunch  
1434375    yay !! this gig is almost over  only and hour ...
262889      welcome ! i m kind of bad about tweeting thou...
1519936     i meant u pwned him !!!! hes just a wimp  lol...
1193415                                less than an hour    
580154                            ugh  i have a belly ache  
799732     apparently farrah fawcett has just passed away...
801412     great weekend with the bffs  freelancing  clea...
1347472                        12 until the final product ! 
1208903                                              thanks 
218226     not a single cat adoption in weeks now  who wa...
1450320                                      yes i am  you ?
13699       yes ! tell us all about your concert coming u...
44718                       all state would like to kill me 
871579      that s sweet of you to ask  world is full of ...
1379344          really 

In [81]:
data.to_csv('inputs/data.csv', index=False)